In [1]:
import os

with open('api_key.txt', 'r') as f:
    key = f.read()

os.environ['GIGACHAT_API_ACCESS_KEY'] = key

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter 
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

file_paths = [
    'docs/laptops1.json',
    'docs/raw_recomm_papers_1.txt',
    'docs/raw_recomm_papers_2.txt',
    'docs/raw_recomm_papers_3.txt'
]

# Load each file and store the results in a list
documents = []
for file_path in file_paths:
    loader = TextLoader(file_path, encoding='utf-8')  # Specify the encoding
    documents.extend(loader.load())


text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(docs, embedding_function)
db

C:\Users\gunte\AppData\Local\Temp\ipykernel_18032\3995579388.py:24: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
C:\Users\gunte\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
retriever = db.as_retriever()
retrieved_docs = retriever.invoke("Ноутбук для учёбы")

retrieved_docs

[Document(metadata={'source': 'docs/laptops1.json'}, page_content='},\n                "Дополнительные характеристики": {\n                    "Гарантия": "12 мес."\n                }\n            }\n        }\n    ]\n]'),
 Document(metadata={'source': 'docs/laptops1.json'}, page_content='},\n                "Операционная система ноутбука": {\n                    "Операционная система": "Free DOS"\n                },\n                "Назначение": {\n                    "Для учебы": "да",\n                    "Для работы": "да"\n                },\n                "Мультимедийные особенности": {\n                    "Для работы": "встроенная",\n                    "Разрешение": "0.9 Мп",\n                    "Тип веб-камеры": "со шторкой",'),
 Document(metadata={'source': 'docs/laptops1.json'}, page_content='},\n                "Операционная система ноутбука": {\n                    "Операционная система": "Free DOS"\n                },\n                "Назначение": {\n               

In [4]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import create_retrieval_chain

In [5]:
llm = GigaChat(
    credentials=os.environ['GIGACHAT_API_ACCESS_KEY'],
    model="GigaChat",
    verify_ssl_certs=False,
    profanity_check=False,
)

prompt = ChatPromptTemplate.from_template(''' Ты - консультант в техническом интернет-магазине \
                                          Ответь на вопрос пользователя. \
Используй при этом информацию из контекста. Если в контексте нет \
информации для ответа, попроси пользователя уточнить необходимые детали.
Контекст: {context}
Вопрос: {input}
Ответ:'''
)


C:\Users\gunte\AppData\Local\Temp\ipykernel_18032\4017348131.py:1: LangChainDeprecationWarning: The class `GigaChat` was deprecated in LangChain 0.3.5 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-gigachat package and should be used instead. To use it run `pip install -U :class:`~langchain-gigachat` and import as `from :class:`~langchain_gigachat import GigaChat``.
  llm = GigaChat(


In [6]:
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
)

In [7]:
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [10]:
question = "Какой лучше ноутбук подойдёт для пожилого человека на пенсии?"


In [13]:
question2 = "Можешь пожалуйста дополнить ответ на предыдущий вопрос, учитывая, что у этого человека также есть внуки, которые могут приходить в гости и иногда играть?"

In [16]:
question3 = "Учитывая всё выше сказанное добавлю, что стоиомсть ноутбука не должна превышать 35 тыс рублей, какие модели ты можешь посоветовать при таком раскладе?"

In [17]:
result = retrieval_chain.invoke({'input': question3})

In [18]:
with open('result.md', 'w', encoding='utf-8') as f:
    f.write(question+'\n\n')
    
    f.write(result['answer'])